# Import libraries

In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from helpers.index_manager import IndexManager
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.dataloader_custom_functions import collate_fn

# Loading data

In [2]:
df_train = pd.read_csv('../data/train-leave2.csv')
df_val = pd.read_csv('../data/val-leave2.csv')
df_test = pd.read_csv('../data/test-leave2.csv')
index_manager = IndexManager()
index_manager.load('../data/index.pkl')

# Creating datasets

In [3]:
train_dataset = NCFDataset(df_train)
val_dataset = NCFDataset(df_val)

dataloader_params = {
    'batch_size': 4096,
    'num_workers': 4,
    'persistent_workers': True,
    'prefetch_factor': 4,
    'pin_memory': True,
    'pin_memory_device': 'cuda',
    'collate_fn': collate_fn,
}

train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
eval_dataloader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# Training model

In [4]:
params = {'factors': 64, 'mlp_user_item_dim': 128, 'learning_rate': 0.001, 'epochs': 50, 'optimizer': 'adagrad', 'dropout': 0.1, 'weight_decay': 0.01, 'loss_fn': 'bce'}

unique_users = index_manager.get_indexed_values('user_id')
unique_items = index_manager.get_indexed_values('item_id')

model = NCFRecommender(unique_users, unique_items, **params)
model.fit(train_dataloader, eval_dataloader)

Epoch 1/50
Train loss: 0.682312, Validation loss: 0.681717
Epoch 2/50
Train loss: 0.667400, Validation loss: 0.673239
Epoch 3/50
Train loss: 0.651131, Validation loss: 0.661895
Epoch 4/50
Train loss: 0.621634, Validation loss: 0.640602
Epoch 5/50
Train loss: 0.565885, Validation loss: 0.621743
Epoch 6/50
Train loss: 0.517191, Validation loss: 0.633076
Epoch 7/50
Train loss: 0.504884, Validation loss: 0.641265
Epoch 8/50
Train loss: 0.503150, Validation loss: 0.643771
Epoch 9/50
Train loss: 0.502600, Validation loss: 0.644508
Epoch 10/50
Train loss: 0.502497, Validation loss: 0.645305
Early stopping triggered after 10 epochs
Training completed!


# Evaluating model

In [5]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate()

Creating ground truth sets...
Generating predictions...
Processing 1 of 121682 users... (0.00%)
Memory usage: 0.411376953125
Increased user batch size from 128 to 141
Increased item batch size from 1024 to 1126
Processing 129 of 121682 users... (0.11%)
Memory usage: 0.411376953125
Increased user batch size from 141 to 155
Increased item batch size from 1126 to 1239
Processing 270 of 121682 users... (0.22%)
Memory usage: 0.411376953125
Increased user batch size from 155 to 170
Increased item batch size from 1239 to 1363
Processing 425 of 121682 users... (0.35%)
Memory usage: 0.411376953125
Increased user batch size from 170 to 187
Increased item batch size from 1363 to 1499
Processing 595 of 121682 users... (0.49%)
Memory usage: 0.411376953125
Increased user batch size from 187 to 206
Increased item batch size from 1499 to 1649
Processing 782 of 121682 users... (0.64%)
Memory usage: 0.411376953125
Increased user batch size from 206 to 227
Increased item batch size from 1649 to 1814
Proc

In [6]:
metrics

{'Hit Ratio@10': 0.041403001265593924,
 'NDCG@10': 0.02005555755203526,
 'Recall@10': 0.041403001265593924,
 'ARP@10': 697353.6873922898,
 'Pop Ratio@10': 1.0}